In [ ]:
!pip install -U datasets huggingface_hub fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0' #user/model_name

quantized format : representing model weights (and sometimes activations) using fewer bits.

it saves memory , increase speed up training and lower hardware requirements.
When you load a model in 4-bit using BitsAndBytes, it means:

-Each weight is stored using just 4 bits

-You only train LoRA adapters, not the full model

In [ ]:
#this says “Load the model weights using 4 bits per value (NF4), and run the math using bfloat16.”
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

This code is loading a quantized large language model (LLM) using Hugging Face's transformers library.

-AutoModelForCausalLM.from_pretrained(...) : hugging face method that loads a pretrained model for causal language modeling(example : next token prediction)

-model_name: pre trained model that we are going to load

-quantization_config=bnb_config : passes the BitsAndBytesConfig object

-device_map="auto" : maps the model to the device (GPU or CPU)

-trust_remote_code=True : Some newer models use custom model classes or
architectures that are defined in their Hugging Face repo. This flag: Allows Hugging Face to download and execute custom Python code for the model architecture.
Without it, you'd get an error for models that require non-standard implementations


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#tokenizer for turning words to tokens
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
lora_config = LoraConfig( #object that tells PEFT how to apply LoRA to your model.
    r = 8, #rank : small number to simplify think , if you achieve good results with lower number (starting 4) keep it
    lora_alpha = 16, #scaling factor that controls how strong the LoRA adapters influence the model's behavior.
    target_modules = ['q_proj', 'v_proj'], #we tell him that we wanna apply it to the matrices realted to query and value
    lora_dropout = 0.05, #dropout applied only to the LoRA layers to improve regularization and prevent overfitting.
    bias = 'none', #Indicates whether to train the bias terms in the model. 'none' means biases are not updated.
    task_type = TaskType.CAUSAL_LM #tells PEFT the type of task.,CAUSAL_LM = causal language modeling, like GPT or any autoregressive model (ex predicting the next token).

)
model = get_peft_model(model, lora_config)


In [ ]:
data = load_dataset('gsm8k', 'main')
data = data['train'].select(range(200))  # select the first 200 examples manually


texts = [

    f"### Instruction:\n{inst}\n### Response:\n{out}"
    for inst, out in zip(batch['question'], batch['answer'])

]

For every question (instruction) and answer (response) in the batch:

It creates a string like:

Instruction:

What is 4 + 5?

Response:

9



In [ ]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['question'], batch['answer'])
    ]
    #This sends the list of texts to the tokenizer:
    tokens = tokenizer(
        texts,
        padding = 'max_length', #pads all sequences to max_length = 256.
        truncation = True, #cuts off anything beyond 256 tokens.
        max_length = 256,
        return_tensors = 'pt' #returns PyTorch tensors, not lists.
    )
    """
    output exemple
      {
    'input_ids': tensor([...]),
    'attention_mask': tensor([...])
    }
    """
    tokens['labels'] = tokens['input_ids'].clone() #for safety

    return tokens


In [ ]:
tokenized_data = data.map(tokenize, batched=True, remove_columns=data.column_names) #tokenize the data

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
#train the data
training_args = TrainingArguments(
    output_dir = './tinyllama-lora-tuned', #output directory of the model
    per_device_train_batch_size = 4, #depends on your gpu
    gradient_accumulation_steps = 4,
    learning_rate = 1e-3,
    num_train_epochs = 50,
    fp16 = True, #half precision to save memory
    logging_steps = 20, #so we don't have to see everything all the time
    save_strategy = 'epoch',
    report_to = 'none',
    remove_unused_columns = False,
    label_names = ["labels"]
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data,
    processing_class = tokenizer
)

In [ ]:
trainer.train()


Step,Training Loss
20,1.912300
40,0.813400
60,0.711900
80,0.626200
100,0.531900
120,0.463600
140,0.390700
160,0.304800
180,0.255000
200,0.192300


TrainOutput(global_step=650, training_loss=0.22725313076606163, metrics={'train_runtime': 1168.9501, 'train_samples_per_second': 8.555, 'train_steps_per_second': 0.556, 'total_flos': 1.590741172224e+16, 'train_loss': 0.22725313076606163, 'epoch': 50.0})

In [ ]:
model.save_pretrained("tinyLlama-lora-tuned-adapter-math")
tokenizer.save_pretrained("tinyLlama-lora-tuned-adapter-math")

('tinyLlama-lora-tuned-adapter-math/tokenizer_config.json',
 'tinyLlama-lora-tuned-adapter-math/special_tokens_map.json',
 'tinyLlama-lora-tuned-adapter-math/chat_template.jinja',
 'tinyLlama-lora-tuned-adapter-math/tokenizer.model',
 'tinyLlama-lora-tuned-adapter-math/added_tokens.json',
 'tinyLlama-lora-tuned-adapter-math/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mv tinyLlama-lora-tuned-adapter-math /content/drive/MyDrive/